In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import argparse
import os, sys
import json
import math
import numpy as np
import copy

from config import config
from model import siMLPe as Model
from model import siMLPe_mod as Model_mod
from datasets.h36m import H36MDataset
from utils.logger import get_logger, print_and_log_info
from utils.pyt_utils import link_file, ensure_dir
from datasets.h36m_eval import H36MEval

from custom_test import test

import torch
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

# cuda setting to make result deterministic
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'

parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
parser.add_argument('--exp-name', type=str, default=None, help='=exp name')
parser.add_argument('--seed', type=int, default=888, help='=seed')
parser.add_argument('--temporal-only', action='store_true', help='=temporal only')
parser.add_argument('--layer-norm-axis', type=str, default='spatial', help='=layernorm axis')
# default is False for 'store_true'
parser.add_argument('--with-normalization', action='store_true', help='=use layernorm')
parser.add_argument('--spatial-fc', action='store_true', help='=use only spatial fc')
parser.add_argument('--num', type=int, default=64, help='=num of blocks')
parser.add_argument('--weight', type=float, default=1., help='=loss weight')

import shlex
argString = '--seed 888 --exp-name baseline.txt --layer-norm-axis spatial --with-normalization --num 48'
args = parser.parse_args(shlex.split(argString))

torch.use_deterministic_algorithms(True)
acc_log = open(args.exp_name, 'a')
torch.manual_seed(args.seed)
writer = SummaryWriter()

config.motion_fc_in.temporal_fc = args.temporal_only
config.motion_fc_out.temporal_fc = args.temporal_only
config.motion_mlp.norm_axis = args.layer_norm_axis
config.motion_mlp.spatial_fc_only = args.spatial_fc
config.motion_mlp.with_normalization = args.with_normalization
config.motion_mlp.num_layers = args.num

acc_log.write(''.join('Seed : ' + str(args.seed) + '\n'))

def get_dct_matrix(N):
	dct_m = np.eye(N)
	for k in np.arange(N):
		for i in np.arange(N):
			w = np.sqrt(2 / N)
			if k == 0:
				w = np.sqrt(1 / N)
			dct_m[k, i] = w * np.cos(np.pi * (i + 1 / 2) * k / N)
	idct_m = np.linalg.inv(dct_m)
	return dct_m, idct_m

# size: (1,T,T)
dct_m,idct_m = get_dct_matrix(config.motion.h36m_input_length_dct)
dct_m = torch.tensor(dct_m).float().cuda().unsqueeze(0)
idct_m = torch.tensor(idct_m).float().cuda().unsqueeze(0)

def update_lr_multistep(nb_iter, total_iter, max_lr, min_lr, optimizer) :
	if nb_iter > 30000:
		current_lr = 1e-5
	else:
		current_lr = 3e-4

	for param_group in optimizer.param_groups:
		param_group["lr"] = current_lr

	return optimizer, current_lr

def gen_velocity(m):
	dm = m[:, 1:] - m[:, :-1]
	return dm

def train_step(h36m_motion_input, h36m_motion_target, model, optimizer, nb_iter, total_iter, max_lr, min_lr) :

	if config.deriv_input:
		b,n,c = h36m_motion_input.shape
		h36m_motion_input_ = h36m_motion_input.clone()
		h36m_motion_input_ = torch.matmul(dct_m[:, :, :config.motion.h36m_input_length], h36m_motion_input_.cuda())
	else:
		h36m_motion_input_ = h36m_motion_input.clone()

	motion_pred = model(h36m_motion_input_.cuda())
	motion_pred = torch.matmul(idct_m[:, :config.motion.h36m_input_length, :], motion_pred)

	if config.deriv_output:
		offset = h36m_motion_input[:, -1:].cuda()
		motion_pred = motion_pred[:, :config.motion.h36m_target_length] + offset
	else:
		motion_pred = motion_pred[:, :config.motion.h36m_target_length]

	# calc losses
	b,n,c = h36m_motion_target.shape
	motion_pred = motion_pred.reshape(b,n,22,3).reshape(-1,3)
	h36m_motion_target = h36m_motion_target.cuda().reshape(b,n,22,3).reshape(-1,3)
	loss = torch.mean(torch.norm(motion_pred - h36m_motion_target, 2, 1))
	# add position loss and velocity loss
	if config.use_relative_loss:
		motion_pred = motion_pred.reshape(b,n,22,3)
		dmotion_pred = gen_velocity(motion_pred)
		motion_gt = h36m_motion_target.reshape(b,n,22,3)
		dmotion_gt = gen_velocity(motion_gt)
		dloss = torch.mean(torch.norm((dmotion_pred - dmotion_gt).reshape(-1,3), 2, 1))
		loss = loss + dloss
	else:
		loss = loss.mean()

	writer.add_scalar('Loss/angle', loss.detach().cpu().numpy(), nb_iter)

	# reset gradients
	optimizer.zero_grad()
	# compute gradients by backpropagation
	loss.backward()
	# update params
	optimizer.step()
	optimizer, current_lr = update_lr_multistep(nb_iter, total_iter, max_lr, min_lr, optimizer)
	writer.add_scalar('LR/train', current_lr, nb_iter)

	return loss.item(), optimizer, current_lr

In [ ]:
# model = Model(config)
model = Model(config)
model.train()
model.cuda()

# dataset = (T-by-C x_in, N-by-C x_out)
config.motion.h36m_target_length = config.motion.h36m_target_length_train
dataset = H36MDataset(config, 'train', config.data_aug)

# separate into batches (input, target) with size (batch_size,T,C) and (batch_size,N,C)
shuffle = True
sampler = None
dataloader = DataLoader(dataset, batch_size=config.batch_size,
						num_workers=config.num_workers, drop_last=True,
						sampler=sampler, shuffle=shuffle, pin_memory=True)

eval_config = copy.deepcopy(config)
eval_config.motion.h36m_target_length = eval_config.motion.h36m_target_length_eval
eval_dataset = H36MEval(eval_config, 'test')

shuffle = False
sampler = None
# separate into batches (input, target) with size (batch_size,T=50,K,3) and (batch_size,N=25,K,3)
eval_dataloader = DataLoader(eval_dataset, batch_size=128,
						num_workers=1, drop_last=False,
						sampler=sampler, shuffle=shuffle, pin_memory=True)


# initialize optimizer
optimizer = torch.optim.Adam(model.parameters(),
							 lr=config.cos_lr_max,
							 weight_decay=config.weight_decay)

ensure_dir(config.snapshot_dir)
logger = get_logger(config.log_file, 'train')
link_file(config.log_file, config.link_log_file)

print_and_log_info(logger, json.dumps(config, indent=4, sort_keys=True))

# continue training from a checkpoint
if config.model_pth is not None :
	state_dict = torch.load(config.model_pth)
	model.load_state_dict(state_dict, strict=True)
	print_and_log_info(logger, "Loading model path from {} ".format(config.model_pth))

Training

In [ ]:
nb_iter = 0
avg_loss = 0
avg_lr = 0

# config.save_every = 5000
# config.cos_lr_total_iters = 5000

# about 4.5 min per 5000 iterations
while (nb_iter + 1) < config.cos_lr_total_iters:

	for (h36m_motion_input, h36m_motion_target) in dataloader:

		loss, optimizer, current_lr = train_step(h36m_motion_input, h36m_motion_target, model, optimizer, nb_iter, config.cos_lr_total_iters, config.cos_lr_max, config.cos_lr_min)
		avg_loss += loss
		avg_lr += current_lr

		if (nb_iter + 1) % config.print_every ==  0 :
			avg_loss = avg_loss / config.print_every
			avg_lr = avg_lr / config.print_every

			print_and_log_info(logger, "Iter {} Summary: ".format(nb_iter + 1))
			print_and_log_info(logger, f"\t lr: {avg_lr} \t Training loss: {avg_loss}")
			avg_loss = 0
			avg_lr = 0

		if (nb_iter + 1) % config.save_every ==  0 :
			torch.save(model.state_dict(), config.snapshot_dir + '/model-iter-' + str(nb_iter + 1) + '.pth')
			model.eval()
			acc_tmp = test(eval_config, model, eval_dataloader)
			print(acc_tmp)
			acc_log.write(''.join(str(nb_iter + 1) + '\n'))
			line = ''
			for ii in acc_tmp:
				line += str(ii) + ' '
			line += '\n'
			acc_log.write(''.join(line))
			model.train()

		if (nb_iter + 1) == config.cos_lr_total_iters :
			break
		nb_iter += 1
	print(nb_iter)
writer.close()